## 0. PSEUDOCODE / OVERVIEW

##### Before this script:
Summed the MapSPAM crop rasters for each year. Code is available at:
<br> https://github.com/gracedoherty/Misc/blob/main/MapSPAM%20batch%20sums.ipynb

<br> Manually renamed the Value of Production rasters from 2005 and 2010 to match naming conventions of the rest. (The VoP files for these years did not need to be summed in the previous script and still had their MapSPAM source file names.)

##### Prep data
Reproject MapSPAM to equal area.

##### Mask to area of interest
Because we are using the clunky tif-xyz-df-gdf zonal stats method, it will reduce processing time to first mask out pixels not in the region of interest.

##### Zonal statistics
Aggregation method: sum.

##### Change over time
Calculate the percent change in agro indicators between the four years of study (2000,2005,2010,2017).

## 1. PREPARE WORKSPACE

### 1.1 Load all packages.

In [1]:
# Built-in:
# dir(), print(), range(), format(), int(), len(), list(), max(), min(), zip(), sorted(), sum(), open(), del, = None, try except, with as, for in, if elif else
# Also: list.append(), list.insert(), list.remove(), count(), startswith(), endswith(), contains(), replace()

import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()
from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from rasterio.enums import Resampling # rasterio.enums.Resampling()
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()

In [8]:
ProjectFolder = os.getcwd()
print(ProjectFolder)

LZFolder = 'Q:\GIS\povertyequity\PTI_Sahel\LivelihoodZones_forPython'
print(LZFolder)

LZPath = os.path.join(LZFolder, 'LZ.gpkg')
print(LZPath)

Q:\GIS\povertyequity\PTI_Sahel\Agriculture
Q:\GIS\povertyequity\PTI_Sahel\LivelihoodZones_forPython
Q:\GIS\povertyequity\PTI_Sahel\LivelihoodZones_forPython\LZ.gpkg


### 1.2 User-defined functions.

In [2]:
def rioStats(InRasterPath, Band = 1):
    out = rasterio.open(InRasterPath)
    stats = []
    band = out.read(Band)
    stats.append({
        'raster': out.name,
        'bands': out.count,
        'data type': out.dtypes,
        'no data value': out.nodatavals,
        'width': out.width,
        'height': out.height,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})
    print("\n", stats)
    
    out = band = None

In [13]:
def MaskByZone(MaskPath, SourceFolder, DestFolder, SourceList = None,
               MaskLayerName = None, dstSRS = 'ESRI:102022'):
    """
    Reduces the size of a raster's valid data cells to vector areas of interest.
    This is useful if the raster data needs to be vectorized later to save space.
    
    The script prepares the vector zones as a list of geometries in the desired
    spatial reference system, then warps each raster in the specified source
    folder to the same SRS. Masking in rasterio then reclassifies any raster cells
    falling outside of a mask polygon as NoData.
    """
    
    ProjSRS = osr.SpatialReference()
    ProjSRS.SetFromUserInput(dstSRS)
    ProjWarp = gdal.WarpOptions(dstSRS = dstSRS)
    
    if SourceList is not None:
        SourceFiles = SourceList
    else:
        SourceFiles = []
        SourceFiles = SourceFiles + [i for i in os.listdir(''.join([SourceFolder, r'/'])) if i.endswith('tif')]
        print(SourceFiles)

    
    ### 1. ASSIGN SPATIAL REFERENCE SYSTEM OF VECTOR MASK AND LOAD GEOMETRIES
    Vector = gpd.read_file(filename=MaskPath, layer=MaskLayerName)
    if Vector.crs != dstSRS:
        if MaskLayerName == None:
            MaskPath = MaskPath + '_temp'
        else:
            MaskLayerName = MaskLayerName + '_temp'
        Vector.to_crs(dstSRS).to_file(filename=MaskPath, layer=MaskLayerName)
    Vector = None # We're reloading the geometries with fiona
    
    with fiona.open(MaskPath, mode="r", layer=MaskLayerName) as Vector:
        MaskGeom = [feature["geometry"] for feature in Vector] # Identify the bounding areas of the mask.
    
    
    ### 2. PREPARE DESTINATION FILES
    for FileName in SourceFiles:
    
        InputRasterPath = os.path.join(ProjectFolder, SourceFolder, FileName)

        TempOutputName = 'Temp_' + FileName
        TempOutputPath = os.path.join(ProjectFolder, DestFolder, TempOutputName)
        FinalOutputName = 'Msk_' + FileName
        FinalOutputPath = os.path.join(ProjectFolder, DestFolder, FinalOutputName)

    ### 3. ASSIGN SPATIAL REFERENCE SYSTEM OF RASTER(S)
        InputRasterObject = gdal.Open(InputRasterPath)
        SourceSRS = osr.SpatialReference(wkt=InputRasterObject.GetProjection())
        print('Source projection: ', SourceSRS.GetAttrValue('projcs'))
        print('Destination projection: ', ProjSRS.GetAttrValue('projcs'))

        if SourceSRS.GetAttrValue('projcs') != ProjSRS.GetAttrValue('projcs'):
            Warp = gdal.Warp(TempOutputPath, # Where to store the warped raster
                         InputRasterObject, # Which raster to warp
                         format='GTiff', 
                         options=ProjWarp) # Reproject to Africa Albers Equal Area Conic
            print('Finished gdal.Warp() for %s. %s \n' % (FileName, time.ctime()))

            Warp = None # Close the files
        else:
            pass
        InputRasterObject = None
        
    ### 4. RECLASSIFY AS NODATA IF OUTSIDE OF SETTLEMENT BUFFER ZONE.
        if exists(TempOutputPath):
            NewInputPath = TempOutputPath 
            print("We warped the data, so we'll use that file for next step.")
        else:
            NewInputPath = InputRasterPath 
            print("We skipped the warp, so we continue to use the source file.")

        with rasterio.open(NewInputPath) as InputRasterObject:
            MaskedOutputRaster, OutTransform = rasterio.mask.mask(
                InputRasterObject, MaskGeom, crop=True) # Anything outside the mask is reclassed to the raster's NoData value.
            OutMetaData = InputRasterObject.meta.copy()
        print('Finished rasterio.mask.mask() for %s. %s \n' % (FileName, time.ctime()))

        OutMetaData.update({"driver": "GTiff",
                         "height": MaskedOutputRaster.shape[1],
                         "width": MaskedOutputRaster.shape[2],
                         "transform": OutTransform})

        with rasterio.open(FinalOutputPath, "w", **OutMetaData) as dest:
            dest.write(MaskedOutputRaster)
        print('Written to file. %s \n' % time.ctime())
        InputRasterObject = None

        if exists(TempOutputPath):
            try:  # Finally, remove the intermediate file from disk
                os.remove(TempOutputPath)
            except OSError:
                pass
            print('Removed intermediate file. %s \n' % time.ctime())
        else:
            pass


    print('\n \n Finished all years in list. %s' % time.ctime())

In [4]:
def BatchZonalStats(FolderName, Zones, 
                    CRS = 'ESRI:102022', 
                    JoinField = 'Sett_ID',
                    StatsWanted = ['count', 'sum', 'mean', 'max', 'min'],
                    SeriesStart = 1999, SeriesEnd = 2015, 
                    AnnualizedFiles = None, VarPrefix = None):
    """
    Normally, we would use numpy to generate a point gdf from the raster's matrix. 
    However, I was running into a lot of memory errors with that method.
    This method uses some extra steps: tif to xyz to df to gdf. But it saves to file
    and deletes intermediate files along the way, circumventing memory issues.
    
    Run MaskByZone() prior to reduce the raster to only your area(s) of interest.
    
    """
    if AnnualizedFiles is None:
        AnnualizedFiles = [i for i in os.listdir(FolderName) if i.endswith('.tif')]
    print(AnnualizedFiles)
    AllSummaries = pd.DataFrame(Zones).drop(columns='geometry')[[JoinField]]
    print(AllSummaries)
    
    if VarPrefix is None:
        VarPrefix = FolderName[:3].upper()
    
    for FileName in AnnualizedFiles:
    ### STEP 1: TIF TO XYZ ###
        print('Loading data for %s. %s \n' % (FileName, time.ctime()))
        
        InputRasterPath = os.path.join(ProjectFolder, FolderName, FileName)
        InputRasterObject = gdal.Open(InputRasterPath)
        XYZOutputPath = FolderName + r'/{}'.format(
            FileName.replace('.tif', '.xyz')) # New file path will be the same as original, but .tif is replaced with .xyz

        # Create an .xyz version of the .tif
        if exists(XYZOutputPath):
            print("Already created xyz file.")
        else:
            print("Creating XYZ (gdal.Translate()).")
            XYZ = gdal.Translate(XYZOutputPath, # Specify a destination path
                                 InputRasterObject, # Input is the masked .tif file
                                 format='XYZ', 
                                 creationOptions=["ADD_HEADER_LINE=YES"])
            print('Finished gdal.Translate() for year %s. %s \n' % (Year, time.ctime()))
            XYZ = None # Reload XYZ as a point geodataframe

        InputRasterObject = None


    ### STEP 2: GENERATE GEODATAFRAME WITH JOIN FIELD ###
        InputXYZ = pd.read_table(XYZOutputPath, delim_whitespace=True)
        InputXYZ = InputXYZ.loc[InputXYZ['Z'] > -1] # Subset to only the features that have a value.
            
        print('Loaded XYZ file as a pandas dataframe. %s \n' % time.ctime())
        ValObject = gpd.GeoDataFrame(InputXYZ,
                                     geometry = gpd.points_from_xy(InputXYZ['X'], InputXYZ['Y']),
                                     crs = CRS)
        print('Created geodataframe from non-NoData points. %s \n' % time.ctime())
        del InputXYZ

        # Sjoin_nearest: No need to group by ADM this time. 
        ValObject_withID = pd.DataFrame(gpd.sjoin_nearest(ValObject, 
                                        Zones, 
                                        how='left')).drop(columns='geometry')[['Z', JoinField]] # No need for max_distance parameter this time. We've already narrowed down to nearby raster cells.

        print('\nJoined zone ID onto vectorized raster cells. %s \n' % time.ctime())
        print(ValObject_withID.sample(10))
        del ValObject

        ValObject_withID.to_csv(''.join([FolderName, r'/', FileName.replace('.tif', '.csv')]))
        print('\nExported as table. %s \n' % time.ctime())


    ### STEP 3: AGGREGATE BY SETTLEMENT AND MERGE ONTO SUMMARIES TABLE ###
        GroupedVals = ValObject_withID[ValObject_withID['Z'].notna()].groupby(JoinField, as_index=False)
        
        # Run this block if the variable is about cloud-free coverage.
        if re.search('cfc', FileName) is not None:
            VariableName = ''.join([VarPrefix, 'cfc_', Sensor, Year])
            AllSummaries = AllSummaries.merge(GroupedVals.mean().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            print('\nCount of cloud-free observations averaged to settlement level, year %s. %s \n' % (Year, time.ctime()))
            
            # Save in-progress results
            AllSummaries.to_csv(os.path.join(ResultsFolder, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
            print(AllSummaries.sample(10))
        
        # Run this block if we're working with the flooded buildings data.
        elif re.search('WSFE', FileName) is not None:
            for BuiltYear in AllStudyYears:
                Grouped_Subset = GroupedVals[GroupedVals['Z'].between(
                    1985, BuiltYear, inclusive=True)] # Inclusive parameter means we include the years 1985 and the named year.
                if 'count' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'ct', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.count().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'sum' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'sum', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.sum().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'mean' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'avg', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.mean().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'max' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'max', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.max().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'min' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'min', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.min().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                print('\nDesired aggregation methods applied to settlement level, year %s. %s \n' % (Year, time.ctime()))

                # Save in-progress results
                AllSummaries.to_csv(os.path.join(ResultsFolder, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
                print(AllSummaries.sample(10))
        
        # Anything else takes the standard aggregation method.
        else:
            if 'count' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'ct', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.count().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'sum' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'sum', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.sum().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'mean' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'avg', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.mean().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'max' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'max', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.max().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'min' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'min', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.min().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            print('\nDesired aggregation methods applied to settlement level, year %s. %s \n' % (Year, time.ctime()))
            
            # Save in-progress results
            AllSummaries.to_csv(os.path.join(ResultsFolder, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
            print(AllSummaries.sample(10))

    
    print('\n\nFinished. All years masked and assigned their nearest settlement. %s' % time.ctime())
    print(AllSummaries.sample(10))
    AllSummaries.to_csv(os.path.join(ResultsFolder, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
    print('Saved to file. %s \n' % time.ctime())

## 2. LOAD DATA

##### ADMs

In [10]:
# The ADMs in the LZ.gpkg are already in our preferred projection.
ADM3, ADM2, ADM1 = gpd.read_file(LZPath, layer='ADM3'), gpd.read_file(LZPath, layer='ADM2'), gpd.read_file(LZPath, layer='ADM1')
ADM3['FID'], ADM2['FID'], ADM1['FID'] = ADM3.index, ADM2.index, ADM1.index
ADM3.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   admin3Name  1433 non-null   object  
 1   admin3Pcod  1433 non-null   object  
 2   admin2Pcod  1433 non-null   object  
 3   admin1Pcod  1433 non-null   object  
 4   admin0Pcod  1433 non-null   object  
 5   ADM3_CODE   1433 non-null   object  
 6   ADM2_CODE   1433 non-null   object  
 7   ADM1_CODE   1433 non-null   object  
 8   geometry    1433 non-null   geometry
 9   FID         1433 non-null   int64   
dtypes: geometry(1), int64(1), object(8)
memory usage: 112.1+ KB


In [12]:
ADMlist = [ADM1, ADM2, ADM3]

##### Agro rasters

In [11]:
SourceFiles = []
SourceFiles = SourceFiles + [i for i in os.listdir('Global_Agro_Sums') if i.endswith('tif')]
SourceFiles

['HarvArea_2000_allTech.tif',
 'HarvArea_2000_Irrigated.tif',
 'HarvArea_2005_allTech.tif',
 'HarvArea_2005_Irrigated.tif',
 'HarvArea_2010_allTech.tif',
 'HarvArea_2010_Irrigated.tif',
 'HarvArea_2017_allTech.tif',
 'HarvArea_2017_Irrigated.tif',
 'PhysArea_2000_allTech.tif',
 'PhysArea_2000_Irrigated.tif',
 'PhysArea_2005_allTech.tif',
 'PhysArea_2005_Irrigated.tif',
 'PhysArea_2010_allTech.tif',
 'PhysArea_2010_Irrigated.tif',
 'PhysArea_2017_allTech.tif',
 'PhysArea_2017_Irrigated.tif',
 'Val_2005_allTech.tif',
 'Val_2005_Irrigated.tif',
 'Val_2010_allTech.tif',
 'Val_2010_Irrigated.tif',
 'Val_2017_allTech.tif',
 'Val_2017_Irrigated.tif']

## 3. MASK TO AREA OF INTEREST

This also reprojects anything that needs to be into the preferred spatial reference.

In [ ]:
# for File in SourceFiles:
#     inRaster = gdal.Open(os.path.join('Global_Agro_Sums', File))
#     outPath = os.path.join('IntermediateFiles', File.replace('.tif', 'EqArea.tif'))
#     warp = gdal.Warp(outPath,inRaster,dstSRS='ESRI:102022')
#     warp = None # Closes the files

In [ ]:
MaskByZone(MaskPath=LZPath, SourceFolder, DestFolder='IntermediateFiles', 
           SourceList = SourceFiles, MaskLayerName = 'ADM1', dstSRS = 'ESRI:102022')

## 4. ZONAL STATISTICS